<a href="https://colab.research.google.com/github/dynamodenis/Artificial_Inteligence/blob/main/Fine_Tuning_LLMs_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [ ]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.48.1
Uninstalling bitsandbytes-0.48.1:
  Successfully uninstalled bitsandbytes-0.48.1
Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [ ]:
# Use Llama pretrained model, load using 4 bit for faster loading
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                                                          bnb_4bit_compute_dtype= getattr(torch, "float16"),
                                                                          bnb_4bit_quant_type="nf4"))

llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Loading the tokenizer

In [ ]:
llama_tockenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                                 trust_remote_code=True)
llama_tockenizer.pad_token = llama_tockenizer.eos_token
llama_tockenizer.padding_side = "right"

## Step 4: Setting the training arguments

In [ ]:
# Define your desired repository name on the Hub

REPO_ID = "dynamo-denis-llama2finetune-medical"
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size=1, max_steps = 100, hub_model_id = REPO_ID )

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split="train"),
                               tokenizer = llama_tockenizer,
                               peft_config = LoraConfig(task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout = 0.1),
                               dataset_text_field="text"
                               )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


## Step 6: Training the model

In [ ]:
# Warning: To run the training you first need to sign up here: https://wandb.ai/authorize?ref=models
# And then you will directly find in the main page your API key which you will enter in the output below.
llama_sft_trainer.train()




The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
wandb: Currently logged in as: dynamodenis254 (dynamodenis254_prism_media_ke) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.7370823669433593, metrics={'train_runtime': 203.8564, 'train_samples_per_second': 0.491, 'train_steps_per_second': 0.491, 'total_flos': 2878908994584576.0, 'train_loss': 1.7370823669433593, 'epoch': 0.014575134819997084})

## Step 7: Chatting with the model

In [ ]:
user_prompt = "Please tell me more about the signs and symptoms of Cancer"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer=llama_tockenizer, max_length=300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]["generated_text"])

Device set to use cuda:0


<s>[INST] Please tell me more about the signs and symptoms of Cancer [/INST]  Cancer is a complex and multifactorial disease that can manifest in different ways, depending on the type of cancer and the individual affected. nobody is immune to cancer, and it is estimated that in 2023, there will be approximately 18.1 million new cases of cancer diagnosed worldwide. Cancer can affect any part of the body, including organs, tissues, and cells. Here are some common signs and symptoms of cancer:

1. Unexplained weight loss: Cancer can cause weight loss due to a variety of factors, including tumor growth, hormonal imbalances, and malnutrition.

2. Fatigue: Feeling tired or weak is a common symptom of cancer, especially in the later stages of the disease.

3. Pain: Pain is a common symptom of cancer, particularly in the later stages of the disease. The pain can be caused by the tumor itself or by the spread of cancer to nearby tissues.

4. Skin changes: Cancer can cause changes in the skin, i

## Step 8: Push the method to hugging face

In [ ]:
from huggingface_hub import login
# Logging to hugging face
login()

# Here we are not going to push the model since its combines the base model and trained model which id too big so we only push the trained weights which is lighter
# llama_model.push_to_hub(repo_name)
# llama_tockenizer.push_to_hub(repo_name)

llama_sft_trainer.push_to_hub(REPO_ID)

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...7842.b200d5607927.12879.0: 100%|##########| 4.18kB / 4.18kB            

  ...8170.b200d5607927.14420.0: 100%|##########| 6.21kB / 6.21kB            

...

  ...56735.b200d5607927.7199.0: 100%|##########| 4.18kB / 4.18kB            

  ...56274.b200d5607927.2184.0: 100%|##########| 5.87kB / 5.87kB            

  ...adapter_model.safetensors:   0%|          |  566kB /  134MB            

  ...6501.b200d5607927.50247.0: 100%|##########| 6.25kB / 6.25kB            

CommitInfo(commit_url='https://huggingface.co/dynamodenis254/dynamo-denis-llama2finetune-medical/commit/76a4aa84e090afa41d6b640fb478812978d1239a', commit_message='dynamo-denis-llama2finetune-medical', commit_description='', oid='76a4aa84e090afa41d6b640fb478812978d1239a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dynamodenis254/dynamo-denis-llama2finetune-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='dynamodenis254/dynamo-denis-llama2finetune-medical'), pr_revision=None, pr_num=None)